# Elevators Logic

Este projeto tem como objetivo implementar algoritmos de busca em um cenário do problema da lógica dos elevadores. O problema consiste em encontrar uma sequência de ações que leve os elevadores a pararem em determinados andares, de acordo com um estado inicial e um estado objetivo.


Esse projeto pode ser visualizado no repositório do <a href="https://github.com/santoguiia/elevators-logic">GitHub</a>

O objetivo principal consiste em abrir os cinco elevadores, cada um contendo uma pessoa, e liberá-las. No entanto, os elevadores só irão se abrir se todos estiverem estacionados em algum andar entre o 21 e o 25, no nível 1 do jogo, ou entre o 21 e o 23, no nível 2. No nível 1, os elevadores começam em andares diferentes: 17, 26, 20, 19 e 31. Já no nível 2, suas posições iniciais são: 20, 20, 22, 24 e 21.

## Algoritmos Implementados

Foram implementados os seguintes algoritmos de busca:

- Busca em Largura (BFS): Explora todos os nós em um mesmo nível antes de avançar para o próximo nível.
- Busca em Profundidade (DFS): Explora um ramo do grafo até encontrar um estado objetivo ou atingir um limite de profundidade.
- Busca em Profundidade Iterativa (IDDFS): Combina a estratégia de busca em profundidade com a busca em largura, aumentando gradualmente o limite de profundidade.
- Busca A*: Utiliza uma função heurística para estimar o custo do caminho até o estado objetivo, combinando o custo do caminho percorrido até o momento com a estimativa do custo restante.

_______

## Estrutura do elevador

In [111]:
from itertools import combinations

def result(state, action):
    new_state = list(state)
    for i, a in enumerate(action):
        new_state[i] = new_positions(state[i], a)
    return tuple(new_state)

def cost(state, action):
    return 1

def goal_test(state):
    return is_goal(state)

def path_cost(c, state1, action, state2):
    return c + cost(state1, action)

def h(node):
    return heuristic(node.state)

def is_valid(state):
    return all(1 <= pos <= 49 for pos in state) and len(set(state)) == 5

def new_positions(position, action):
    new_position = position + action
    if 1 <= new_position <= 49:
        return new_position
    return position

def get_successors(state):
    successors = []
    actions = [(8, 8), (-13, -13), (8, -13), (-13, 8)]
    for (e1, e2) in combinations(range(5), 2):
        for action in actions:
            new_state = list(state)
            new_state[e1] = new_positions(state[e1], action[0])
            new_state[e2] = new_positions(state[e2], action[1])
            if new_state[e1] != state[e1] or new_state[e2] != state[e2]:
                successors.append(tuple(new_state))
    return successors

def print_solution(solution):
    for node in solution:
        print(node.state)

## Seleção de dificuldade

### Escolha do Nível
Selecione o nível que deseja que os algoritmos de busca tentem resolver. Os níveis correspondem aos diferentes cenários propostos no problema dos elevadores.

In [80]:
# level 1
initial_state = (17, 26, 20, 19, 31)

def is_goal(state):
    return all(21 <= pos <= 25 for pos in state)


In [12]:
# level 2
initial_state = (20, 20, 22, 24, 21)

def is_goal(state):
    return all(21 <= pos <= 23 for pos in state)



### Heuristica
Selecione a Heuristica que deseja usar.

In [144]:
# Heuristica 1 - Distancia minima de cada posicao ate o target mais proximo
def heuristic(state):
    return sum(min(abs(pos - target) for target in range(21, 26)) for pos in state)

In [130]:
# Heuristica 2 - Uso de energia
def heuristic(state):
    energy_cost_per_floor = 0.1  # Define o custo de energia por andar percorrido (em kWh)
    target_floors = range(21, 26)  # Andares de destino possíveis

    total_energy_cost = 0
    for pos in state:
        for target in target_floors:
            total_energy_cost += abs(pos - target) * energy_cost_per_floor

    return total_energy_cost

In [126]:
# Heurística 3 - Eficiência de Uso dos Elevadores:
def heuristic(state):
    efficiency_score = 0
    for floor in state:
        efficiency_score += abs(floor - 25)  # Avalia a distância de cada elevador ao andar de destino mais frequente (25)
    return efficiency_score


## Algoritmos de Busca

### Busca em Largura

Na estratégia de busca em largura, após a expansão do nó raiz, todos os seus sucessores são explorados. Em seguida, os sucessores desses sucessores são considerados. O conceito-chave aqui é o uso de uma fronteira em forma de pilha, onde o primeiro elemento a entrar é o primeiro a sair. Uma analogia comum é a fila de um supermercado, onde as pessoas são atendidas na ordem em que chegaram.

In [52]:
from collections import deque

def bfs_search(initial_state):
    frontier = deque([(initial_state, 0)])
    explored = set()

    while frontier:
        current_state, current_cost = frontier.popleft()

        if is_goal(current_state):
            return current_state, current_cost

        if current_state not in explored:
            explored.add(current_state)
            successors = get_successors(current_state)

            for state in successors:
                if state not in explored:
                    cost = current_cost + 1
                    frontier.append((state, cost))

    return None, None

bfs_search(initial_state)

((25, 21, 23, 22, 21), 7)

### Busca em Profundidade
Na Busca por Profundidade, o algoritmo inicia pela raiz do grafo e explora o primeiro sucessor encontrado. Ele então avança para o primeiro sucessor desse sucessor, continuando assim até alcançar o ponto mais profundo, onde não existem mais sucessores. Para isso, utiliza-se uma estrutura de dados conhecida como pilha, na qual o último elemento inserido é o primeiro a ser retirado. 

In [ ]:
def dfs_search(initial_state):
    frontier = [(initial_state, 0)]
    explored = set()

    while frontier:
        current_state, current_cost = frontier.pop()

        if is_goal(current_state):
            return current_state, current_cost

        if current_state not in explored:
            explored.add(current_state)
            successors = get_successors(current_state)

            # Reverse the successors to prioritize deeper exploration
            successors.reverse()

            for state in successors:
                if state not in explored:
                    cost = current_cost + 1  # Uniform cost
                    frontier.append((state, cost))

    return None, None  # No solution found

dfs_search(initial_state)


### Busca em profundidade iterativo
Neste algoritmo, também é adotada uma abordagem com limite de profundidade. No entanto, a cada tentativa de resolver o problema sem sucesso, o algoritmo aumenta o limite de profundidade em uma unidade.

In [141]:
def iddfs_search(initial_state):
    depth = 0
    while True:
        result, cost = dls_search(initial_state, depth)
        if result:
            return result, cost
        depth += 1

def dls_search(initial_state, depth_limit):
    frontier = deque([(initial_state, 0)])
    explored = set()

    while frontier:
        current_state, current_cost = frontier.pop()

        if is_goal(current_state):
            return current_state, current_cost

        if current_state not in explored and current_cost < depth_limit:
            explored.add(current_state)
            successors = get_successors(current_state)

            for state in successors:
                if state not in explored:
                    cost = current_cost + 1  # Uniform cost
                    frontier.append((state, cost))

    return None, None  # No solution found

iddfs_search(initial_state)


((25, 21, 23, 22, 21), 7)

### Busca A*
Nesse algoritmo, o A* utiliza uma função de estimativa, que consiste na soma do custo do caminho percorrido até o momento com uma função heurística \(f(n) = g(n) + h(n)\). Essa abordagem pode ser interpretada como o custo total da solução, priorizando a expansão do nó com o menor valor de \(f(n)\) primeiro.

In [143]:
# best-first search algorithm that uses a heuristic to determine the order in which nodes are expanded

import heapq

def a_star_search(initial_state):
    frontier = []
    heapq.heappush(frontier, (heuristic(initial_state), initial_state, 0))
    explored = set()
    
    while frontier:
        _, current_state, current_cost = heapq.heappop(frontier)
        
        if is_goal(current_state):
            return current_state, current_cost
        
        if current_state not in explored:
            explored.add(current_state)
            successors = get_successors(current_state)
            
            for state in successors:
                if state not in explored:
                    cost = current_cost + 1  # Uniform cost
                    heapq.heappush(frontier, (cost + heuristic(state), state, cost))
                    
    return None, None  # No solution found

# Initial state with the elevators stopped on floors
initial_state = (17, 26, 20, 19, 31)
a_star_search(initial_state)


((25, 21, 23, 22, 21), 7)